In [1]:
from google.colab import drive
drive.mount("/content/d")

Mounted at /content/d


In [23]:
from IPython.display import clear_output
!pip install jupyter-dash
clear_output()

In [26]:
import json
import random
import requests
import time
import os

import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

from collections import defaultdict
import pandas as pd

# DATA COLLECTION

In [14]:
start_time = time.time()
print(f"PROCESS STARTED...")

def getPage(page = 0):
    params = {
        'text': 'NAME:Программист',
        'area': 1,
        'page': page,
        'per_page': 100 
    }
    
    req = requests.get('https://api.hh.ru/vacancies', params) 
    data = req.content.decode() 
    req.close()
    return data

for page in range(0, 20):
    
    jsObj = json.loads(getPage(page))
    nextFileName = 'd/MyDrive/hhApi/docs/pagination/{}.json'.format(len(os.listdir('d/MyDrive/hhApi/docs/pagination')))
    
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
    
    if (jsObj['pages'] - page) <= 1:
        break
    time.sleep(0.25)


for fl in os.listdir('d/MyDrive/hhApi/docs/pagination'):
    
    f = open('d/MyDrive/hhApi/docs/pagination/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    
    jsonObj = json.loads(jsonText)
    for v in jsonObj['items']:
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
        
        fileName = 'd/MyDrive/hhApi/docs/vacancies/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
        
        time.sleep(random.uniform(0.3, 1))
        
print(f"...PROCESS COMPLETED, time: {int(time.time() - start_time)}s")

...PROCESS COMPLETED


In [4]:
fPath = "d/MyDrive/hhApi/docs/vacancies"

for filename in os.listdir("d/MyDrive/hhApi/docs/vacancies"):
    os.rename(os.path.join(fPath,filename), os.path.join(fPath,"1_"+filename))

In [16]:
!mv d/MyDrive/hhApi/docs/vacancies/* d/MyDrive/hhApi/docs/stack/

In [28]:
len(os.listdir("d/MyDrive/hhApi/docs/stack"))

4360

In [29]:
len(os.listdir("d/MyDrive/hhApi/docs/vacancies"))

0

# DATA PROCESSING

In [21]:

res = pd.DataFrame(
    dtype=object,
    columns = [
        "name", "city", "salary_from", "salary_to", "salary_currency", "salary_is_gross",
        "location_address", "experience", "schedule", "employment_type",
        "skills_required", "specializations", "professional_roles", "employer_name",
        "employer_is_trusted", "working_days", "working_time_intervals",
        "working_time_modes", "languages"
    ]    
)

In [33]:
for f in j

{'errors': [{'value': 'captcha_required',
   'captcha_url': 'https://hh.ru/account/captcha?state=pxvcxBozfu7ry7R4QCetFpeicJ2Ml-ur_701s7jX2mZxCzHziryEFFjVgnI-vznk7kbvQSRzYITqyYULknPgPy2Gi-QYy3Z-bi2YBfq8rBk7oCQhTIav06s7vtDFqMtv',
   'type': 'captcha_required'}],
 'request_id': '16718786695656329146af11470e2378'}

In [31]:
start_time = time.time()
print(f"PROCESS STARTED...")

toMove = []

for f in os.listdir("d/MyDrive/hhApi/docs/stack/"):
    # print(f)
    try:
        j = json.load(open("d/MyDrive/hhApi/docs/stack/" + f, "rb"))
        name = j["name"] if "name" in j.keys() else "-1"
        city = j["area"]["name"] if "area" in j.keys() else "-1"
        salary_from = j["salary"]["from"] if j["salary"] != None else "-1"  #all(["salary" in j.keys(), "from" in j["salary"].keys()]) else "-1"
        salary_to = j["salary"]["to"] if j["salary"] != None else "-1" #all(["salary" in j.keys(), "to" in j["salary"].keys()]) else "-1"
        salary_currency = j["salary"]["currency"] if j["salary"] != None else "-1" #all(["salary" in j.keys(), "currency" in j["salary"].keys()]) else "-1"
        salary_is_gross = j["salary"]["gross"] if j["salary"] != None else "-1" #all(["salary" in j.keys(), "gross" in j["salary"].keys()]) else "-1"
        location_address = j["address"] if "address" in j.keys() else "-1"
        experience = j["experience"]["id"] if "experience" in j.keys() else "-1"
        schedule = j["schedule"]["id"] if "schedule" in j.keys() else "-1"
        employment_type = j["employment"]["id"] if "employment" in j.keys() else "-1"
        skills_required = [skill["name"] for skill in j['key_skills']] if "key_skills" in j.keys() else "-1"
        specializations = [sp["name"] for sp in j["specializations"]] if "specializations" in j.keys() else "-1"
        professional_roles = [rl["name"] for rl in j['professional_roles']] if "professional_roles" in j.keys() else "-1"
        employer_name = j["employer"]["name"] if "employer" in j.keys() else "-1"
        employer_is_trusted = j["employer"]["trusted"] if all(["employer" in j.keys(), "trusted" in j["employer"].keys()]) else "-1"
        working_days = j["working_days"]
        working_time_intervals = j["working_time_intervals"]
        working_time_modes = j["working_time_modes"]
        languages = j["languages"]

        res.loc[len(res)]  = (
            name, city, salary_from, salary_to, salary_currency, salary_is_gross, 
            location_address, experience, schedule, employment_type, skills_required,
            specializations, professional_roles, employer_name, employer_is_trusted,
            working_days, working_time_intervals, working_time_modes, languages)
        
    except:
        # os.remove("d/MyDrive/hhApi/docs/stack/" + f)
        # continue
        break

    # res.loc[len(res)] = (
    #     j["name"], j["area"]["name"], j['salary']["from"], j['salary']["to"], j['salary']["currency"], 
    #     j['salary']["gross"], j["address"], j['experience']["id"], j['schedule']["id"], j["employment"]["id"],
    #     [skill["name"] for skill in j['key_skills']], [sp["name"] for sp in j["specializations"]],
    #     [rl["name"] for rl in j['professional_roles']], j["employer"]["name"], j["employer"]["trusted"], 
    #     j["working_days"], j["working_time_intervals"], j["working_time_modes"], j["languages"]
    # )
    
print(f"...PROCESS COMPLETED, time: {int(time.time() - start_time)}s")
res

PROCESS STARTED...
...PROCESS COMPLETED, time: 0s


,name,city,salary_from,salary_to,salary_currency,salary_is_gross,location_address,experience,schedule,employment_type,skills_required,specializations,professional_roles,employer_name,employer_is_trusted,working_days,working_time_intervals,working_time_modes,languages
0,"TeamLead GO Developer (API Gateway, Terraform ...",Москва,-1,-1,-1,-1,None,between3And6,fullDay,full,[],"[Программирование, Разработка, Системы управле...","[Программист, разработчик]",СБЕР,True,[],[],[],[]
1,Инженер-программист в информационно - аналитич...,Москва,80000,None,RUR,False,"{'city': 'Москва', 'street': 'улица Пресненски...",between3And6,fullDay,full,"[MS SQL Server, MS SQL, SQL, Работа с большим ...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",ФГБУ Федеральный центр тестирования,True,[],[],[],"[{'id': 'eng', 'name': 'Английский', 'level': ..."
2,Программист 1C (разработка),Москва,-1,-1,-1,-1,"{'city': 'Москва', 'street': '1-й переулок Тру...",between3And6,fullDay,full,[],"[Программирование, Разработка, Системы управле...","[Программист, разработчик]",COLDY,True,[],[],[],[]
3,Программист,Москва,-1,-1,-1,-1,"{'city': 'Москва', 'street': 'Сходненская улиц...",between1And3,fullDay,full,"[Python, Linux, SQL, Git, PostgreSQL, C/C++, A...","[Розничная торговля, Прямые продажи, Менеджер ...",[Другое],HARTUNG,True,[],[],[],[]
4,Ведущий программист / разработчик Linux,Москва,300000,None,RUR,False,"{'city': None, 'street': None, 'building': Non...",between3And6,fullDay,full,"[Разработка ПО, Linux, Python, PostgreSQL, СУБД]","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",МД Информационные Системы,True,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,Golang-developer в Payment Processing,Москва,-1,-1,-1,-1,None,between3And6,fullDay,full,"[Golang, PostgreSQL, gRPC, Redis, RabbitMQ]","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",WILDBERRIES,True,[],[],[],[]
715,Разработчик .NET (POS),Москва,-1,-1,-1,-1,None,between3And6,remote,full,"[C#, .NET, POS, Windows Forms, WinForms, WPF, ...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",ICL Services,True,[],[],[],[]
716,Ведущий инженер-программист ПЛИС (VHDL / Verilog),Санкт-Петербург,-1,-1,-1,-1,"{'city': 'Санкт-Петербург', 'street': 'Новосел...",noExperience,fullDay,full,"[Git, Atlassian Jira, Linux, VHDL, Verilog HDL...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Радар ММС,True,[],[],[],[]
717,"Разработчик Go, Логистика, Тарификация магистр...",Санкт-Петербург,-1,-1,-1,-1,"{'city': 'Санкт-Петербург', 'street': 'Херсонс...",between3And6,fullDay,full,"[Golang, Linux, Английский язык, PostgreSQL, A...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Ozon,True,[],[],[],[]


In [27]:
res.shape

(360, 19)

In [ ]:
j = json.load(open("d/MyDrive/hhApi/docs/vacancies/73121079.json", "rb"))
j.keys()

dict_keys(['id', 'premium', 'billing_type', 'relations', 'name', 'insider_interview', 'response_letter_required', 'area', 'salary', 'type', 'address', 'allow_messages', 'experience', 'schedule', 'employment', 'department', 'contacts', 'description', 'branded_description', 'vacancy_constructor_template', 'key_skills', 'accept_handicapped', 'accept_kids', 'archived', 'response_url', 'specializations', 'professional_roles', 'code', 'hidden', 'quick_responses_allowed', 'driver_license_types', 'accept_incomplete_resumes', 'employer', 'published_at', 'created_at', 'initial_created_at', 'negotiations_url', 'suitable_resumes_url', 'apply_alternate_url', 'has_test', 'test', 'alternate_url', 'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary', 'languages'])

In [ ]:
print(res[(res.salary_from != "-1")]&(res.salary_to != "-1")

,name,city,salary_from,salary_to,salary_currency,salary_is_gross,location_address,experience,schedule,employment_type,skills_required,specializations,professional_roles,employer_name,employer_is_trusted,working_days,working_time_intervals,working_time_modes,languages
3,Senior PHP Developer,Москва,None,200000,RUR,True,"{'city': 'Москва', 'street': 'Пресненская набе...",between3And6,fullDay,full,"[HTML, CSS, PHP, Лидерство, HTML/CSS, Database...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Ра Сын Гир,False,[],[],[],[]
4,Программист 1С,Москва,None,300000,RUR,False,None,between1And3,fullDay,full,[],"[Программирование, Разработка, Системы управле...","[Программист, разработчик]","Hi, Rockits!",True,[],[],[],[]
11,Bitrix разработчик,Москва,90000,None,RUR,False,"{'city': 'Москва', 'street': 'Профсоюзная улиц...",between1And3,fullDay,full,"[MySQL, PHP, Git, PostgreSQL]","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",idaproject,True,[],[],[],[]
12,Программист 1C,Москва,160000,180000,RUR,False,"{'city': 'Москва', 'street': 'улица Льва Толст...",between3And6,remote,full,[],"[Программирование, Разработка, Системы управле...","[Программист, разработчик]",TOM TAILOR,True,[],[],[],[]
15,Senior Backend разработчик PHP Symphony,Москва,None,350000,RUR,False,"{'city': 'Москва', 'street': 'улица Земляной В...",moreThan6,fullDay,full,"[PostgreSQL, Docker, Архитектурная деятельност...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Инженерный Центр Информационно-аналитических с...,True,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8365,Mobile-разработчик (React/React Native),Санкт-Петербург,170000,None,RUR,False,"{'city': 'Санкт-Петербург', 'street': 'Литовск...",between3And6,fullDay,full,"[JSON, Мобильные приложения, React, XML]","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",САТУРН - строительные материалы,True,[],[],[],[]
8371,Наставник курса “Профессия Разработчик на Python”,Санкт-Петербург,40000,80000,RUR,False,None,between3And6,fullDay,full,"[Python, SQL, Django Framework, Flask, Linux, ...","[Гуманитарные науки, Языки, Преподавание]","[Учитель, преподаватель, педагог]",Skyeng,True,[],[],[],[]
8373,Наставник на онлайн-курс «Профессия Frontend-р...,Санкт-Петербург,20000,None,RUR,False,None,between1And3,remote,part,"[Ответственность, Angular, Frontend, Удаленная...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Айтилогия,True,[],[],[],[]
8374,"Программист 1С (разработчик 1C, удаленно)",Санкт-Петербург,70000,140000,RUR,False,"{'city': 'Санкт-Петербург', 'street': 'улица М...",between3And6,remote,part,"[1С: Управление Торговлей, 1С программирование...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Аркада,True,[],[],"[{'id': 'start_after_sixteen', 'name': 'Можно ...",[]


In [ ]:
jsonObj

{'id': '74157780',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Архитектор на custom development',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1',
  'name': 'Москва',
  'url': 'https://api.hh.ru/areas/1?host=hh.ru'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': False,
 'experience': {'id': 'between1And3', 'name': 'От 1 года до 3 лет'},
 'schedule': {'id': 'fullDay', 'name': 'Полный день'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': None,
 'description': '<p><strong>Требования к кандидату:</strong></p> <ul> <li>Опыт работы в роли системного архитектора или тимлида в проектах по разработке корпоративного ПО и информационных систем основанных на микросервисной архитектуре в крупных компаниях со сложным ИТ-ландшафтом. Количество проектов не менее 3-х.</li> <li>Опыт работы на проектах с классич

# DATA RESEARCH

In [25]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
app.run_server(mode='inline')

# CONCLUSIONS

<PLACEHOLDER>